In [69]:
# extract text from resumes, output to jsonl

import re
import json
import spacy as spacy
import srsly
'''
Make a "rehearsal" update to the models in the pipeline, to prevent forgetting. Rehearsal updates run an initial copy of the model over some data, and update the model so its current predictions are more like the initial ones. This is useful for keeping a pretrained model on-track, even if you're updating it with a smaller set of examples.
'''

# avoids JSONDecodeError due to malformed json file
data = [json.loads(line)
        for line in open("../../data/annotated/resumedata.json", "r", encoding="utf-8")]

content = []
for k in data:
    dct = {
        "text": re.sub(r"\s+", " ", k["content"])
    }
    content.append(dct)

srsly.write_json("../../data/resume.jsonl", content)


In [124]:
# create a silver annotated dataset
'''
Rehearsal updates run an initial copy of the model over some data
'''
nlp = spacy.load("en_core_web_md")

with open("../../data/resume.jsonl", "r", encoding="utf-8") as f1:
    resume = json.load(f1)

    annotated = list()
    for k in resume:
        doc = nlp(k["text"])
        for sent in doc.sents:
            labels = list()
            for e in sent.ents:
                labels.append([e.start_char, e.end_char, e.label_])
            if labels:
                spacy_entry = (sent.text, {"entities": labels})
                annotated.append(spacy_entry)

    srsly.write_json("../../data/annotated/rehearse_silver_spacy.jsonl", annotated)


In [155]:
# convert echr training data to jsonl file so that we can train a spacy model
from spacy.tokens import DocBin

doc_bin = DocBin().from_disk("../../data/annotated/train.spacy")
examples = []
for doc in doc_bin.get_docs(nlp.vocab):
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))

    spacy_entry = (doc.text, {"entities": entities})
    examples.append(spacy_entry)

    srsly.write_json("../../data/annotated/echr_train_spacy.jsonl", examples)


In [ ]:
# convert echr training data to jsonl file so that we can test a spacy model

doc_bin = DocBin().from_disk("../../data/annotated/dev.spacy")
examples = []
for doc in doc_bin.get_docs(nlp.vocab):
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))

    spacy_entry = (doc.text, {"entities": entities})
    examples.append(spacy_entry)

    srsly.write_json("../../data/annotated/echr_dev_spacy.jsonl", examples)

In [ ]:
# train the first model on echr data
import json
from spacy.training.example import Example
from collections import Counter
import spacy
nlp = spacy.load("en_core_web_md", enable="ner")
ner = nlp.get_pipe('ner')

# from GH https://github.com/explosion/spaCy/issues/7161
output_dir = "../../data/models/spacy/output"

with open('../../data/annotated/echr_train_spacy.jsonl', "r", encoding="utf-8") as f1:
    train = json.load(f1)

    # fetch and add labels
    count_per_entity_new = Counter()
    for _, annotations in train:
        for ent in annotations.get("entities"):
           count_per_entity_new[ent[2]] +=1

    for k in count_per_entity_new:
        ner.add_label(k)

optimizer = nlp.create_optimizer()
for itn in range(2):
    for raw_text,entity_offsets in train:
        doc = nlp.make_doc(raw_text)
        example = Example.from_dict(doc,entity_offsets)
        nlp.update([example],sgd=optimizer)

nlp.to_disk(output_dir)


KeyboardInterrupt: 

In [19]:
# testing the model
!python -m spacy benchmark accuracy "../../../data/models/spacy/output" "../../../data/annotated/dev.spacy"

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   29.27 
NER R   39.50 
NER F   33.63 
SPEED   22339 


=============================== NER (per type) ===============================

                  P       R       F
CARDINAL       0.00    0.00    0.00
GPE            0.68    0.36    0.48
ORG            9.32   10.29    9.78
LAW            0.00    0.00    0.00
NORP           0.00    0.00    0.00
PERSON        18.95   20.78   19.82
DATE          89.70   82.87   86.15
LOC            0.00    0.00    0.00
PRODUCT        0.00    0.00    0.00
ORDINAL        0.00    0.00    0.00
DEM            0.00    0.00    0.00
TIME           0.00    0.00    0.00
QUANTITY       0.00    0.00    0.00
MONEY          0.00    0.00    0.00
PERCENT        0.00    0.00    0.00
LANGUAGE       0.00    0.00    0.00
WORK_OF_ART    0.00    0.00    0.00
FAC            0.00    0.00    0.00
EVENT          0.0

In [164]:
import random
from collections import Counter
# step 2
# ref: https://github.com/explosion/spaCy/discussions/10041

with open('../../data/annotated/rehearse_silver_sent_spacy.jsonl', "r") as f2:
    rehearse = json.load(f2)


    # fetch and add labels
    count_per_entity_new = Counter()
    for _, annotations in rehearse:
        for ent in annotations.get("entities"):
           count_per_entity_new[ent[2]] +=1

    for k in count_per_entity_new:
        ner.add_label(k)

    nlp_incremental_try = spacy.load("../../data/models/spacy/md/model-last")
    ner = nlp_incremental_try.get_pipe('ner')

# '''
# Make a "rehearsal" update to the models in the pipeline, to prevent forgetting. Rehearsal updates run an initial copy of the model over some data, and update the model so its current predictions are more like the initial ones. This is useful for keeping a pretrained model on-track, even if you're updating it with a smaller set of examples.
#
# rehearsal, collect samples of text you want the models to retain performance on, and call nlp.rehearse() with a batch of Example objects.
# raw_batch = [Example.from_dict(nlp.make_doc(text), {}) for text in next(raw_text_batches)]
# '''
# # https://spacy.io/api/pipe#rehearse
#
    optimizer = nlp_incremental_try.resume_training()
    for itn in range(20):
        random.shuffle(rehearse)
        for raw_text, entity_offsets in rehearse:
            doc = nlp_incremental_try.make_doc(raw_text)
            example = Example.from_dict(doc,entity_offsets)
            nlp_incremental_try.rehearse([example],sgd=optimizer)
    nlp.to_disk("../../data/models/spacy/rehearse")


/Users/bpayne/code/data-science/pii-redaction/venv/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "30-Jan 2021 WORK EXPERIENCE Technology Consultant ..." with entities "[[47, 58, 'DATE'], [97, 109, 'ORG'], [110, 133, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/bpayne/code/data-science/pii-redaction/venv/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "CRM Sales ( CRM 7.01, CRM 7.02, CRM 7.03) -Feb '20..." with entities "[[19, 23, 'CARDINAL'], [29, 33, 'CARDINAL'], [51, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/bpayne/code/data-science/pii-reda

In [165]:
!python -m spacy benchmark accuracy "../../data/models/spacy/rehearse" "../../data/annotated/rehearse_silver_sent_test.jsonl"

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/Users/bpayne/code/data-science/pii-redaction/venv/lib/python3.10/site-packages/spacy/training/corpus.py:102: UserWarning: [W090] Could not locate any .spacy files in path '../../data/annotated/rehearse_silver_sent_test.jsonl'.
  warnings.warn(Warnings.W090.format(path=orig_path, format=file_type))

================================== Results ==================================

TOK     -
NER P   -
NER R   -
NER F   -
SPEED   0



In [122]:
TRAIN_DATA = [
    ('Who is Kofi Annan?',{
    'entities':[(7,18,'PERSON')]
    }),
    ('Who is Steve Jobs?',{
    'entities':[(7,17,'PERSON')]
    }),
    ('I like London and Berlin.',{
    'entities':[(7,13,'LOC'),(18,24,'LOC')]
    })
]
print(type(TRAIN_DATA))


<class 'list'>
